# Top 10 arrival airports in the world in 2013 (using the bookings file)

• Arrival airport is the column arr_port. It is the IATA code for the airport. 

• To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

• Print the top 10 arrival airports in the standard output, including the number of passengers.

• Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

• Bonus point: Solve this problem using pandas (instead of any other approach)

## Action plan

1. In 'Booking' sample:
    - Selection of 2013 observations by filtering by column 'year'
    - Group by column 'arr_port' and sum 'pax' for each airport
    - Sort results in descending order using *sort_values()*
    - Print top 10 results by using *head()*
        

2. In 'Booking' sample by using chunks:
    - Create iterator object using parameter *nrows* to run it as a sample.
    - Create 'all_chunks' variable as an empty list.
    - Adapt our 'Booking' sample code to be run in every chunk loop:
        - Filter each chunk by year 2013
        - Group by 'arr_port' and sum 'pax' for each one.
        - Append results to 'all_chunks' variable.
    - Concat all chunks results.
    - *Groupby()* again, *sort_values()* and *head()*


3. Replicate the process for the whole dataset (eliminate *nrows* parameter and adapt chunksize) and save results in *pax_per_airport_2013*


4. Bonus point:
    - Import airport using *pd.read_json()* info from: https://raw.githubusercontent.com/mwgg/Airports/master/airports.json'
    - Traspose the resulting dataframe
    - Select 'iata' and 'city' columns
    - Merge to *pax_per_airport_2013* by 'iata' and 'arr_port' column
    - Drop 'iata' column in the resulting dataframe as it is a duplicate of 'arr_port'


## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline

## Data Path in my computer

In [2]:
!ls /home/dsc/Data/challenge/

bookings.csv.bz2		 searches_with_bookings.csv
bookings_without_duplicates.csv  searches_without_duplicates.csv
searches.csv.bz2


## Step 1: Let's start by working with a sample of our dataset

We will use the sample we have created in Exercise 1

In [3]:
bookings_sample = pd.read_csv('bookings.sample.csv.bz2',compression='bz2', sep='^')

In [4]:
bookings_sample.shape

(999, 38)

In [5]:
bookings_sample.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [6]:
bookings_sample.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [7]:
bookings_sample_2013 = bookings_sample[bookings_sample['year'] == 2013]
bookings_sample_2013

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2013-03-23 00:00:00,1P,US,bfafedb1505531ad8e70e176941a7ad0,804deeaddf16e6a93d36e5d046040d60,be83fd657625d53f58c5d3070cdb41b8,2013-03-06 00:00:00,5570,0,SLC,...,SLCMSN,NV,K,Y,2013-05-20 14:00:00,2013-05-20 18:00:55,-1,2013,3,NULL
995,2013-03-23 00:00:00,1P,US,bfafedb1505531ad8e70e176941a7ad0,804deeaddf16e6a93d36e5d046040d60,be83fd657625d53f58c5d3070cdb41b8,2013-03-06 00:00:00,5570,0,SLC,...,SLCMSN,NV,K,Y,2013-05-20 14:00:00,2013-05-20 18:00:55,1,2013,3,NULL
996,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,691f15622668e671f51ba033ea904398,2013-03-06 00:00:00,4664,0,SNA,...,SNASFO,FK,V,Y,2013-03-17 13:31:00,2013-03-17 15:01:12,1,2013,3,NULL
997,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,691f15622668e671f51ba033ea904398,2013-03-06 00:00:00,4664,0,SNA,...,SFOSNA,FK,W,Y,2013-03-20 17:45:00,2013-03-20 19:15:12,1,2013,3,NULL


In [8]:
bookings_sample_2013.groupby('arr_port')['pax'].sum()

arr_port
ACC         6
ADD         2
AGP         6
ALA         2
ALC         4
           ..
YWG         4
YXE         1
YYC         0
YYZ         2
ZRH        -1
Name: pax, Length: 201, dtype: int64

In [9]:
bookings_sample_2013.groupby('arr_port')['pax'].sum().sort_values(ascending=False).head(10)

arr_port
DCA         50
JFK         14
FLL         13
PVG         13
SVO         12
LGA         12
CGK         12
MIA         11
LAS         11
BOS         11
Name: pax, dtype: int64

## Step 2: Let's try it now with a sample and by using chunks

We will do it by using chunks and by selecting only 'year', 'arr_port' and 'pax' columns in order to reduce the file size and the time of execution.

In [10]:
chksize = 100

In [11]:
reader = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2',compression='bz2',sep='^', usecols=['year','arr_port','pax'], nrows=999, iterator=True, chunksize=chksize)
all_chunks= []
chunk_counter=0

for df in reader:
    df = df[df['year'] == 2013]
    result_chunk = df.groupby('arr_port')['pax'].sum()
    all_chunks.append(result_chunk)
    chunk_counter+=1
    print(chunk_counter)

pax_per_airport_2013 = pd.concat(all_chunks)

1
2
3
4
5
6
7
8
9
10


In [12]:
type(all_chunks)

list

In [13]:
type(pax_per_airport_2013)

pandas.core.series.Series

In [14]:
pax_per_airport_2013.shape

(350,)

In [15]:
pax_per_airport_2013.head()

arr_port
ALA         2
ALG         0
AYT         3
BLR         1
BWI         6
Name: pax, dtype: int64

In [16]:
pax_per_airport_2013.reset_index()

,arr_port,pax
0,ALA,2
1,ALG,0
2,AYT,3
3,BLR,1
4,BWI,6
...,...,...
345,SFO,2
346,SJO,0
347,SVO,4
348,TLV,1


In [17]:
pax_per_airport_2013.reset_index().groupby('arr_port')['pax'].sum().sort_values(ascending=False).head(10)

arr_port
DCA         50
JFK         14
FLL         13
PVG         13
SVO         12
LGA         12
CGK         12
MIA         11
LAS         11
BOS         11
Name: pax, dtype: int64

## Step 3: Let's try it now with the whole dataset

We will do it by using chunks and by selecting only year, 'arr_port' and 'pax' columns in order to reduce the file size and the time of execution.

In [18]:
chksize = 500000

In [19]:
%%time

reader = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2',compression='bz2',sep='^', usecols=['year','arr_port','pax'], iterator=True, chunksize=chksize)
all_chunks= []
chunk_counter=0

for df in reader:
    df = df[df['year'] == 2013]
    result_chunk = df.groupby('arr_port')['pax'].sum()
    all_chunks.append(result_chunk)
    chunk_counter+=1
    print(chunk_counter)

pax_per_airport_2013 = pd.concat(all_chunks)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
CPU times: user 2min 35s, sys: 1.71 s, total: 2min 37s
Wall time: 2min 35s


In [20]:
pax_per_airport_2013.reset_index().groupby('arr_port')['pax'].sum().sort_values(ascending=False).head(10)

arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
Name: pax, dtype: float64

## Step 4: Bonus Point

Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github

In [21]:
airports_info = pd.read_json('https://raw.githubusercontent.com/mwgg/Airports/master/airports.json')

In [22]:
airports_info.head()

,00AK,00AL,00AZ,00CA,00CO,00FA,00FL,00GA,00ID,00IL,...,ZYMD,ZYMH,ZYQQ,ZYTH,ZYTL,ZYTN,ZYTX,ZYXC,ZYYJ,ZYYY
icao,00AK,00AL,00AZ,00CA,00CO,00FA,00FL,00GA,00ID,00IL,...,ZYMD,ZYMH,ZYQQ,ZYTH,ZYTL,ZYTN,ZYTX,ZYXC,ZYYJ,ZYYY
iata,,,,,,,,,,,...,MDG,OHE,NDG,,DLC,TNH,SHE,XEN,YNJ,
name,Lowell Field,Epps Airpark,Cordes Airport,Goldstone /Gts/ Airport,Cass Field,Grass Patch Airport,River Oak Airport,Lt World Airport,Delta Shores Airport,Hammer Airport,...,Mudanjiang Hailang International Airport,Gu-Lian Airport,Qiqihar Sanjiazi Airport,Tahe Airport,Zhoushuizi Airport,Tonghua Sanyuanpu Airport,Taoxian Airport,Xingcheng Air Base,Yanji Chaoyangchuan Airport,Shenyang Dongta Airport
city,Anchor Point,Harvest,Cordes,Barstow,Briggsdale,Bushnell,Okeechobee,Lithonia,Clark Fork,Polo,...,Mudanjiang,Mohe,Qiqihar,Tahe,Dalian,Tonghua,Shenyang,,Yanji,Shenyang
state,Alaska,Alabama,Arizona,California,Colorado,Florida,Florida,Georgia,Idaho,Illinois,...,Heilongjiang,Heilongjiang,Heilongjiang,Heilongjiang,Liaoning,Jilin,Liaoning,Liaoning,Jilin,Liaoning


In [23]:
airports_info = airports_info.T

In [24]:
airports_city = airports_info[['iata','city']]

In [25]:
pax_per_airport_city_2013 = pax_per_airport_2013.merge(airports_city,left_on=['arr_port'],right_on=['iata'], how='left')
pax_per_airport_city_2013

AttributeError: 'Series' object has no attribute 'merge'

Let's investigate why merging on 'arr_port' and 'iata' is not working:

In [26]:
airports_city[airports_city['iata'] == 'JFK']

,iata,city
KJFK,JFK,New York


In [27]:
pax_per_airport_2013 = pax_per_airport_2013.reset_index()

In [28]:
pax_per_airport_2013.head()

,arr_port,pax
0,AAE,16.0
1,AAL,18.0
2,AAN,0.0
3,AAQ,126.0
4,AAR,34.0


In [29]:
pax_per_airport_2013[pax_per_airport_2013['arr_port']=='AAE']

,arr_port,pax


In [30]:
pax_per_airport_2013['arr_port'].dtype

dtype('O')

In [31]:
pax_per_airport_2013['arr_port'] = pax_per_airport_2013['arr_port'].astype(str)

In [32]:
pax_per_airport_2013[pax_per_airport_2013['arr_port']=='AAE']

,arr_port,pax


In [33]:
pax_per_airport_2013.arr_port[0]

'AAE     '

And, here is the problem. Merging is not working because the values of 'arr_port' has whitespaces. 

In [34]:
pax_per_airport_2013['arr_port'] = pax_per_airport_2013['arr_port'].str.strip()

In [35]:
pax_per_airport_2013[pax_per_airport_2013['arr_port']=='JFK']

,arr_port,pax
802,JFK,3490.0
2875,JFK,3137.0
4929,JFK,3490.0
7002,JFK,3137.0
9056,JFK,3490.0
11129,JFK,3137.0
13183,JFK,3490.0
15256,JFK,3137.0
17310,JFK,3490.0
19383,JFK,3137.0


Let's try to merge again:

In [36]:
pax_per_airport_city_2013 = pax_per_airport_2013.merge(airports_city,left_on=['arr_port'],right_on=['iata'], how='left')

In [37]:
pax_per_airport_city_2013

,arr_port,pax,iata,city
0,AAE,16.0,AAE,Annabah
1,AAL,18.0,AAL,Aalborg
2,AAN,0.0,AAN,Al Ain
3,AAQ,126.0,AAQ,Anapa
4,AAR,34.0,AAR,Aarhus
...,...,...,...,...
41330,LAS,4.0,LAS,Las Vegas
41331,MAA,2.0,MAA,Chennai
41332,MEL,2.0,MEL,Melbourne
41333,ORY,2.0,ORY,Paris


In [38]:
pax_per_airport_city_2013 = pax_per_airport_city_2013[['arr_port','city','pax']]
pax_per_airport_city_2013

,arr_port,city,pax
0,AAE,Annabah,16.0
1,AAL,Aalborg,18.0
2,AAN,Al Ain,0.0
3,AAQ,Anapa,126.0
4,AAR,Aarhus,34.0
...,...,...,...
41330,LAS,Las Vegas,4.0
41331,MAA,Chennai,2.0
41332,MEL,Melbourne,2.0
41333,ORY,Paris,2.0


In [39]:
pax_per_airport_city_2013.groupby(['arr_port','city'])[['pax']].sum().sort_values(by='pax',ascending=False).head(10)

,,pax
arr_port,city,
LHR,London,88809.0
MCO,Orlando,70930.0
LAX,Los Angeles,70530.0
LAS,Las Vegas,69630.0
JFK,New York,66270.0
CDG,Paris,64490.0
BKK,Bangkok,59460.0
MIA,Miami,58150.0
SFO,San Francisco,58000.0
